In [27]:

# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [28]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [30]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels_NN = reformat(train_dataset, train_labels)
valid_dataset, valid_labels_NN = reformat(valid_dataset, valid_labels)
test_dataset, test_labels_NN = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels_NN.shape)
print('Validation set', valid_dataset.shape, valid_labels_NN.shape)
print('Test set', test_dataset.shape, test_labels_NN.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [31]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Problem 1
Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). The right amount of regularization should improve your validation / test accuracy

In [32]:
# Logistic Regression
# Dataset containing duplicates
import random
import sklearn
from sklearn.linear_model import LogisticRegression

In [33]:

train_dataset_lin = train_dataset
test_dataset_lin = test_dataset
valid_dataset_lin = valid_dataset

score_train=[]
score_test=[]
score_valid=[]
train_sample = [ 50, 100, 1000 , 5000 ]
for samplesize in train_sample:
    logreg = LogisticRegression(penalty ="l2") #C=1e5 C=0.001
    idx = random.sample(range(len(train_labels)),samplesize)
    logreg.fit(train_dataset_lin[idx,:], train_labels[idx])
              
    res = logreg.predict(train_dataset_lin)
    score_train.append( sklearn.metrics.accuracy_score(train_labels, res) )

    res = logreg.predict(test_dataset_lin)
    score_test.append( sklearn.metrics.accuracy_score(test_labels, res) )

    res = logreg.predict(valid_dataset_lin)
    score_valid.append( sklearn.metrics.accuracy_score(valid_labels, res) )
    
    
print("Train score ",score_train)
print("Test score",score_test)
print("Valid score",score_valid)

Train score  [0.62558499999999995, 0.70928500000000005, 0.75572499999999998, 0.78030500000000003]
Test score [0.69569999999999999, 0.78059999999999996, 0.83340000000000003, 0.8478]
Valid score [0.62329999999999997, 0.71309999999999996, 0.75729999999999997, 0.7752]


In [41]:
beta = 0.01

DNN with Logistic Classifier with l2 regularization

In [42]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels_NN[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    regularizer = tf.nn.l2_loss(weights)
    loss = tf.reduce_mean(loss + beta * regularizer)
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
   

In [43]:
num_steps = 801
# num_steps = 101

def accuracy(predictions, labels):
#     print(np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)))
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels_NN[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels_NN))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels_NN))

Initialized
Loss at step 0: 49.179230
Training accuracy: 7.6%
Validation accuracy: 10.2%
Loss at step 100: 11.839695
Training accuracy: 74.3%
Validation accuracy: 72.9%
Loss at step 200: 4.494623
Training accuracy: 79.2%
Validation accuracy: 77.2%
Loss at step 300: 1.985639
Training accuracy: 82.3%
Validation accuracy: 80.1%
Loss at step 400: 1.131721
Training accuracy: 83.7%
Validation accuracy: 81.2%
Loss at step 500: 0.838350
Training accuracy: 84.3%
Validation accuracy: 82.0%
Loss at step 600: 0.736394
Training accuracy: 84.5%
Validation accuracy: 82.2%
Loss at step 700: 0.700633
Training accuracy: 84.5%
Validation accuracy: 82.1%
Loss at step 800: 0.687996
Training accuracy: 84.5%
Validation accuracy: 82.2%
Test accuracy: 88.9%


with regularization accuracy improved to 88.9 from 82.7

    DNN with 1 hidden layer on notMnist SGD l2 reguarization

In [44]:
batch_size = 128
num_of_nodes = 1024
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  


    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_of_nodes]))
    biases_1= tf.Variable(tf.zeros([num_of_nodes]))
    weights_L = tf.Variable(
        tf.truncated_normal([num_of_nodes, num_labels]))
    biases_L= tf.Variable(tf.zeros([num_labels]))
                           
    
    # Training computation.
    
    relu_layer_hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
    logits = tf.matmul(relu_layer_hidden1, weights_L) + biases_L
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  


    regularizer = tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_L)
    loss = tf.reduce_mean(loss + beta * regularizer)
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_L) + biases_L)
    test_prediction =  tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_L) + biases_L)


In [45]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels_NN[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels_NN))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_NN))

Initialized
Minibatch loss at step 0: 3512.934570
Minibatch accuracy: 7.0%
Validation accuracy: 23.6%
Minibatch loss at step 500: 21.154699
Minibatch accuracy: 87.5%
Validation accuracy: 83.5%
Minibatch loss at step 1000: 0.888603
Minibatch accuracy: 80.5%
Validation accuracy: 82.7%
Minibatch loss at step 1500: 0.802426
Minibatch accuracy: 83.6%
Validation accuracy: 82.8%
Minibatch loss at step 2000: 0.888120
Minibatch accuracy: 79.7%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 0.683008
Minibatch accuracy: 85.2%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 0.666697
Minibatch accuracy: 84.4%
Validation accuracy: 83.3%
Test accuracy: 89.8%


Problem 2
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

In [47]:
num_steps = 3001
train_dataset_2 = train_dataset[:500, :]
train_labels_2 = train_labels_NN[:500]


with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_2[offset:(offset + batch_size), :]
        batch_labels = train_labels_2[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels_NN))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_NN))

Initialized
Minibatch loss at step 0: 3474.408203
Minibatch accuracy: 5.5%
Validation accuracy: 41.0%
Minibatch loss at step 500: 21.036211
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 1000: 0.472407
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Minibatch loss at step 1500: 0.298038
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Minibatch loss at step 2000: 0.280724
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Minibatch loss at step 2500: 0.273897
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Minibatch loss at step 3000: 0.272107
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Test accuracy: 86.0%


As we can see the mini batch accuracy is 100% while valiadation accuracy keeps on decreasing. this show overfitting

Problem 3
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides nn.dropout() for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

In [49]:
batch_size = 128
num_of_nodes = 1024
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  


    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_of_nodes]))
    biases_1= tf.Variable(tf.zeros([num_of_nodes]))
    weights_L = tf.Variable(
        tf.truncated_normal([num_of_nodes, num_labels]))
    biases_L= tf.Variable(tf.zeros([num_labels]))
                           
    
    # Training computation.
    
    relu_layer_hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
    #Dropout on hidden layer
    keep_prob = tf.placeholder("float")
    relu_layer_hidden1_dropout = tf.nn.dropout(relu_layer_hidden1,keep_prob)
    
    logits = tf.matmul(relu_layer_hidden1_dropout, weights_L) + biases_L
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  


    regularizer = tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_L)
    loss = tf.reduce_mean(loss + beta * regularizer)
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    #Drop out should be only on training, not on validation or test set 
    # Predictions for the training
    train_prediction = tf.nn.softmax(logits)
    
    # Predictions for validation, and test data.
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_L) + biases_L)
    test_prediction =  tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_L) + biases_L)


In [51]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels_NN[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels_NN)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels_NN)))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3636.27880859375
Minibatch accuracy: 10.2
Validation accuracy: 22.1
Minibatch loss at step 500: 21.323780059814453
Minibatch accuracy: 85.9
Validation accuracy: 83.8
Minibatch loss at step 1000: 0.9715715646743774
Minibatch accuracy: 81.2
Validation accuracy: 83.1
Minibatch loss at step 1500: 0.8942941427230835
Minibatch accuracy: 79.7
Validation accuracy: 82.7
Minibatch loss at step 2000: 1.0060890913009644
Minibatch accuracy: 75.8
Validation accuracy: 82.5
Minibatch loss at step 2500: 0.7372539043426514
Minibatch accuracy: 86.7
Validation accuracy: 82.5
Minibatch loss at step 3000: 0.7739846706390381
Minibatch accuracy: 82.8
Validation accuracy: 83.0
Test accuracy: 89.5


<b>Extreme Overfitting</b>



In [56]:
num_steps = 3001
train_dataset_2 = train_dataset[:500, :]
train_labels_2 = train_labels_NN[:500]


with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_2[offset:(offset + batch_size), :]
        batch_labels = train_labels_2[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels_NN))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_NN))

Initialized
Minibatch loss at step 0: 3632.745117
Minibatch accuracy: 8.6%
Validation accuracy: 32.1%
Minibatch loss at step 500: 21.186647
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 0.520697
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Minibatch loss at step 1500: 0.329082
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Minibatch loss at step 2000: 0.305369
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Minibatch loss at step 2500: 0.311038
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Minibatch loss at step 3000: 0.309852
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Test accuracy: 85.8%


Problem 4
Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

global_step = tf.Variable(0)  # count the number of steps taken.
learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

In [58]:
import math as math

In [63]:
batch_size = 128
beta = 0.001

hidden_nodes_1 = 1024
hidden_nodes_2 = int(hidden_nodes_1 * 0.5)
hidden_nodes_3 = int(hidden_nodes_1 * np.power(0.5, 2))
hidden_nodes_4 = int(hidden_nodes_1 * np.power(0.5, 3))
hidden_nodes_5 = int(hidden_nodes_1 * np.power(0.5, 4))

graph = tf.Graph()
with graph.as_default():

    '''Input Data'''
    # For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    '''Variables'''
    # Hidden RELU layer 1
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1], stddev=math.sqrt(2.0/(image_size*image_size))))
    biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))

    # Hidden RELU layer 2
    weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], stddev=math.sqrt(2.0/hidden_nodes_1)))
    biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
    
    # Hidden RELU layer 3
    weights_3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, hidden_nodes_3], stddev=math.sqrt(2.0/hidden_nodes_2)))
    biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))
    
    # Hidden RELU layer 4
    weights_4 = tf.Variable(tf.truncated_normal([hidden_nodes_3, hidden_nodes_4], stddev=math.sqrt(2.0/hidden_nodes_3)))
    biases_4 = tf.Variable(tf.zeros([hidden_nodes_4]))
    
    # Hidden RELU layer 5
    weights_5 = tf.Variable(tf.truncated_normal([hidden_nodes_4, hidden_nodes_5], stddev=math.sqrt(2.0/hidden_nodes_4)))
    biases_5 = tf.Variable(tf.zeros([hidden_nodes_5]))
    
    # Output layer
    weights_6 = tf.Variable(tf.truncated_normal([hidden_nodes_5, num_labels], stddev=math.sqrt(2.0/hidden_nodes_5)))
    biases_6 = tf.Variable(tf.zeros([num_labels]))
    
    '''Training computation'''
    
    # Hidden RELU layer 1
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden_layer_1 = tf.nn.relu(logits_1)
    # Dropout on hidden layer: RELU layer
    keep_prob = tf.placeholder("float")
    hidden_layer_1_dropout = tf.nn.dropout(hidden_layer_1, keep_prob)
    
    
    # Hidden RELU layer 2
    logits_2 = tf.matmul(hidden_layer_1_dropout, weights_2) + biases_2
    hidden_layer_2 = tf.nn.relu(logits_2)
    # Dropout on hidden layer: RELU layer
    hidden_layer_2_dropout = tf.nn.dropout(hidden_layer_2, keep_prob)
    
    # Hidden RELU layer 3
    logits_3 = tf.matmul(hidden_layer_2_dropout, weights_3) + biases_3
    hidden_layer_3 = tf.nn.relu(logits_3)
    # Dropout on hidden layer: RELU layer
    hidden_layer_3_dropout = tf.nn.dropout(hidden_layer_3, keep_prob)
    
    # Hidden RELU layer 4
    logits_4 = tf.matmul(hidden_layer_3_dropout, weights_4) + biases_4
    hidden_layer_4 = tf.nn.relu(logits_4)
    # Dropout on hidden layer: RELU layer

    hidden_layer_4_dropout = tf.nn.dropout(hidden_layer_4, keep_prob)
    
    # Hidden RELU layer 5
    logits_5 = tf.matmul(hidden_layer_4_dropout, weights_5) + biases_5
    hidden_layer_5 = tf.nn.relu(logits_5)
    # Dropout on hidden layer: RELU layer
    hidden_layer_5_dropout = tf.nn.dropout(hidden_layer_5, keep_prob)
    
    # Output layer
    logits_6 = tf.matmul(hidden_layer_5_dropout, weights_6) + biases_6 
    
    # Normal loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels =tf_train_labels,logits=logits_6))
    # Loss function with L2 Regularization with decaying learning rate beta=0.5
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + \
                   tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4) + \
                   tf.nn.l2_loss(weights_5) + tf.nn.l2_loss(weights_6)
    loss = tf.reduce_mean(loss + beta * regularizers)

    '''Optimizer'''
    # Decaying learning rate
    global_step = tf.Variable(0)  # count the number of steps taken.
    start_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_6)
    
    # Predictions for validation 
    valid_logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    valid_relu_1 = tf.nn.relu(valid_logits_1)
    
    valid_logits_2 = tf.matmul(valid_relu_1, weights_2) + biases_2
    valid_relu_2 = tf.nn.relu(valid_logits_2)
    
    valid_logits_3 = tf.matmul(valid_relu_2, weights_3) + biases_3
    valid_relu_3 = tf.nn.relu(valid_logits_3)
    
    valid_logits_4 = tf.matmul(valid_relu_3, weights_4) + biases_4
    valid_relu_4 = tf.nn.relu(valid_logits_4)
    
    valid_logits_5 = tf.matmul(valid_relu_4, weights_5) + biases_5
    valid_relu_5 = tf.nn.relu(valid_logits_5)
    
    valid_logits_6 = tf.matmul(valid_relu_5, weights_6) + biases_6
    
    valid_prediction = tf.nn.softmax(valid_logits_6)
    
    # Predictions for test
    test_logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    test_relu_1 = tf.nn.relu(test_logits_1)
    
    test_logits_2 = tf.matmul(test_relu_1, weights_2) + biases_2
    test_relu_2 = tf.nn.relu(test_logits_2)
    
    test_logits_3 = tf.matmul(test_relu_2, weights_3) + biases_3
    test_relu_3 = tf.nn.relu(test_logits_3)
    
    test_logits_4 = tf.matmul(test_relu_3, weights_4) + biases_4
    test_relu_4 = tf.nn.relu(test_logits_4)
    
    test_logits_5 = tf.matmul(test_relu_4, weights_5) + biases_5
    test_relu_5 = tf.nn.relu(test_logits_5)
    
    test_logits_6 = tf.matmul(test_relu_5, weights_6) + biases_6
    
    test_prediction = tf.nn.softmax(test_logits_6)

In [65]:
num_steps = 10000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels_NN[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels_NN)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels_NN)))

Initialized
Minibatch loss at step 0: 5.345345973968506
Minibatch accuracy: 8.6
Validation accuracy: 10.9
Minibatch loss at step 500: 1.6823533773422241
Minibatch accuracy: 85.9
Validation accuracy: 82.4
Minibatch loss at step 1000: 1.3475247621536255
Minibatch accuracy: 82.8
Validation accuracy: 83.4
Minibatch loss at step 1500: 1.327812910079956
Minibatch accuracy: 80.5
Validation accuracy: 84.5
Minibatch loss at step 2000: 1.303621768951416
Minibatch accuracy: 73.4
Validation accuracy: 84.5
Minibatch loss at step 2500: 0.859140157699585
Minibatch accuracy: 88.3
Validation accuracy: 84.4
Minibatch loss at step 3000: 0.8516361117362976
Minibatch accuracy: 83.6
Validation accuracy: 84.9
Minibatch loss at step 3500: 0.9944417476654053
Minibatch accuracy: 82.0
Validation accuracy: 85.2
Minibatch loss at step 4000: 0.9481039047241211
Minibatch accuracy: 80.5
Validation accuracy: 85.1
Minibatch loss at step 4500: 0.8265371322631836
Minibatch accuracy: 85.9
Validation accuracy: 86.2
Minibat